In [ ]:
import time
import numpy as np

from geo import *
from Thymio import Thymio
import motion_control
import filtering
import global_navigation


# -- Global Settings --
THYMIO_PORT = "COM6"
THYMIO_REFRESH_RATE = 1.0
G_verbose = True
S_camera_interval = 1000 #ms
S_motion_interval = 1000
S_track_interval = 0.1 #s
G_track_timer = time.time()

S_epsilon_dis = 1
S_epsilon_theta = 0.1

S_stablize_filter_steps = 10
# -- Controllers --
G_th = Thymio.serial(port=THYMIO_PORT, refreshing_rate=THYMIO_REFRESH_RATE)
G_mc = motion_control.MotionController(G_th, S_motion_interval,verbose=False)
G_mc.timer = time.time()

pre_state = np.array([0, 0, 0]).reshape(-1, 1) # initial state
pre_cov = np.ones([3, 3]) * 0.03 # initial covariance
G_filter = filtering.KF(pre_state, pre_cov, qx=0.1, qy=0.1, qtheta=0.3, rl=0.1, rr=0.1, b=0.08)
G_filter.timer = time.time()

In [ ]:
def localizate():
    """Track Where Thymio is"""
    # 3. Localization 
    # 3.1 odometer
    dsl, dsr = G_mc.get_displacement()
    G_filter.kalman_filter(dsr, dsl)
    
    thymio_state = G_filter.get_state()
    return thymio_state

In [ ]:
# h, w = 30, 40
# rmap = GridMap(h, w, 0.01)
# rmap.set_goal(Pos(h-1, w-1))
# rmap.set_start(Pos(0,0))
# import random
# obslist = [Pos(random.randint(8,h-8),random.randint(8,w-8)) for _ in range(3)]
# rmap.set_obs(obslist)

# Goal_state = State(Pos(h-1,w-1), 0.0)

# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [20, 10]

# # planner
# ppr = global_navigation.PathPlanner(rmap,path_simplification=True, plot=True,neighbor=8, method="A*")

In [ ]:
#print(Goal_state)

In [ ]:
#path = ppr.plan()
Global_path = [State(Pos(0.1, 0.0), 0.0),State(Pos(0.2, 0.1), 0.0)]

In [ ]:
#Global_path = global_navigation.assign_ori(path, Goal_state.ori)

In [ ]:
for i in range(3):
    print(G_mc.thymio.get_var_array("prox.horizontal"))
    time.sleep(0.6)
G_mc.thymio.set_var("motor.left.target", 0)
G_mc.thymio.set_var("motor.right.target", 0)

In [ ]:

try:
    while True:
        starter = time.time()
        # 3. Localization
        Thymio_state = localizate()
        #print(F"Thymio:{Thymio_state}, Target:{Global_path[0]}")
        # 2.2.1 Finished?
        if len(Global_path) == 0:
            if G_verbose:
                print("Terminate Reached!")
            break
        # 4. Follow the path
        if starter - G_track_timer > S_track_interval:
            print(F"Thymio:{Thymio_state}, Target:{Global_path[0]}")
            reached = G_mc.path_tracking(Global_path[0], Thymio_state)
            if reached:
                Global_path = Global_path[1:]
            G_track_timer = starter
                # assume Global_path is not empty because of 2.2.1
        #loop_time = time.time() - starter
        #if G_verbose and len(Global_path) > 0:
        #    print(F"Thymio:{Thymio_state}, Target:{Global_path[0]}, {loop_time*1000 :.0f}ms")
        time.sleep(0.01)
finally:
    G_mc.stop()
    G_mc.close()